In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML


# empty dataframe with column names A B C D Y1 Y2
df_full = pd.DataFrame(columns=['A', 'B', 'C', 'D', 'Y1', 'Y2'])

# infinite run statistics:
for a,b,c,d in np.ndindex(3,2,2,2):
    df_full = pd.concat([df_full, pd.DataFrame({'A': [a], 'B': [b], 'C': [c], 'D': [d], 
                                              'Y1': [np.minimum((0.5*a + 0.3*b)*c, d)], 
                                              'Y2': [np.where(a == 0, 0, np.where(a == 1, 0.1*a + 0.2*b + 0.3*c + 0.4*d, a - d))]})], 
                                              ignore_index=True)

In [2]:
def display_side_by_side(dfs:list, captions:list):
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))


# pd.set_option("display.max_rows", None)
# display(HTML("<div style='height: 200px'>" + df_partial.style.render() + "</div>"))

In [3]:
# now lets say you find two datasets that only cover a subset of the variables
df_A = pd.DataFrame(columns=['A', 'B', 'Y1'])
df_B = pd.DataFrame(columns=['B', 'C', 'D', 'Y2'])


# df_A = df_full[['A', 'B', 'Y1']] without D = 0
df_A = pd.concat([df_A, df_full.loc[df_full.D != 0, ['A', 'B', 'Y1']]], ignore_index=True)

# df_B = df_full[['B', 'C', 'D', 'Y2']] without A = 2
df_B = pd.concat([df_B, df_full.loc[df_full.A != 2, ['B', 'C', 'D', 'Y2']]], ignore_index=True)

# df_partial = df_A.merge(df_B, on='B', how='outer')

In [4]:
# display_side_by_side([df_full.sort_values(by='D'), df_full.sort_values(by='A'), df_A, df_B], 
#                      ['Full by D', 'Full by A', 'df_A: df_full.D != 0', 'df_B: df_full.A != 2'])

display_side_by_side([df_full, df_B, df_A], 
                     ['df_full', 'df_B: df_full.A != 2', 'df_A: df_full.D != 0'])

,A,B,C,D,Y1,Y2
0,0,0,0,0,0.000000,0.0
1,0,0,0,1,0.000000,0.0
2,0,0,1,0,0.000000,0.0
3,0,0,1,1,0.000000,0.0
4,0,1,0,0,0.000000,0.0
5,0,1,0,1,0.000000,0.0
6,0,1,1,0,0.000000,0.0
7,0,1,1,1,0.300000,0.0
8,1,0,0,0,0.000000,0.1
9,1,0,0,1,0.000000,0.5


In [5]:
# # try to estimate via gurobi:
# import gurobipy as gp
# from gurobipy import GRB


# ##### we need all our variables discrete:
# # binning Y1 from df_A:
# df_full_disc = df_full.copy()
# df_full_disc['Y1'] = pd.cut(df_full['Y1'], bins=2, labels=[0, 1])
# df_full_disc['Y2'] = pd.cut(df_full['Y2'], bins=2, labels=[0, 1])

# df_A_disc = df_A.copy()
# df_B_disc = df_B.copy()
# df_A_disc['Y1'] = pd.cut(df_A['Y1'], bins=2, labels=[0, 1])
# df_B_disc['Y2'] = pd.cut(df_B['Y2'], bins=2, labels=[0, 1])

# ####

# m = gp.Model()
# # model.params.NonConvex = 2
# card_hidden = 16
# cardY1, cardY2 = 2, 2
# (cardA, cardB, cardC, cardD) = (3,2,2,2)

# # target
# target = m.addMVar(shape = (cardY1, cardY2, cardA, cardB, cardC, cardD, card_hidden), 
#                              vtype=GRB.INTEGER, name="f(Y1, Y2, A,B,C,D|hidden)", lb=0, ub=1)
# # set up so that it's a deterministic function, gives the target dataset value for Y1 if Y1 = 1 and Y2 = 0, and gives Y2 if Y1 = 0 and Y2 = 1

# for a,b,c,d, hidden in np.ndindex(cardA, cardB, cardC, cardD, card_hidden):
#     m.addConstr(target[1, 1, a, b, c, d, hidden] == GRB.UNDEFINED, name=f"f(1, 1, {a},{b},{c},{d}, {hidden}) = UNDEFINED")
#     m.addConstr(target[0, 0, a, b, c, d, hidden] == GRB.UNDEFINED, name=f"f(1, 1, {a},{b},{c},{d}, {hidden}) = UNDEFINED")

# # using our partial data:
# # for a, b, y1 in np.ndindex(cardA, cardB, cardY1):
# #     for hidden in range(card_hidden):
# #         m.addConstr(target[y1, 0, a, b, :, :, hidden] == df_A_disc.loc[(df_A_disc.A == a) & (df_A_disc.B == b), 'Y1'].values[0] or df_A_disc.loc[(df_A_disc.A == a) & (df_A_disc.B == b), 'Y1'].values[1], 
# #             name=f"f({y1}, 0, {a},{b},:,:) = {df_A_disc.loc[(df_A_disc.A == a) & (df_A_disc.B == b), 'Y1'].values[0]} or {df_A_disc.loc[(df_A_disc.A == a) & (df_A_disc.B == b), 'Y1'].values[1]}")

                    
# m.update()
# m.getConstrs()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09


[<gurobi.Constr f(1, 1, 0,0,0,0, 0) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 0) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 1) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 1) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 2) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 2) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 3) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 3) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 4) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 4) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 5) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 5) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 6) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 6) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 7) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 7) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 8) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 8) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 9) = UNDEFINED>,
 <gurobi.Constr f(1, 1, 0,0,0,0, 9) = UNDEFINED>,


In [11]:
# # target[Y1, Y2, A, B, C, D, hidden]
# # target[1, 0, 1, 1, :, :, hidden] # means find Y1 for A=1, B=1, Y1=1, C, D, hidden = hidden

# # lst = df_A_disc.loc[(df_A_disc.A == a) & (df_A_disc.B == b), 'Y1'] #.values[1]
# # for i in range(len(lst)):
# #     print(lst.values[i])

# # a, b, y1 = 1, 1, 1
# # m.addConstr(target[y1, 0, a, b, :, :, hidden][0,0] == gp.or_(0,1),  name='constraint')
# # target[y1, 0, a, b, :, :, hidden][0,0] == gp.or_(0,1)


# a, b, y1 = 1, 1, 1

# z1 = m.addVar(vtype=GRB.BINARY, name="z1")
# z2 = m.addVar(vtype=GRB.BINARY, name="z2")
# z3 = m.addVar(vtype=GRB.BINARY, name="z3")

# # Define a large constant M
# M = 1000

# m.update()
# # Add big-M constraints
# m.addConstr(target[y1, 0, a, b, :, :, hidden][0,0] >= 1 - M * (1 - z1), name= f'f({y1}, 0, {a},{b},0,0) >= 1 - M * (1 - z1) = {1 - M * (1 - z1)}')
# m.addConstr(target[y1, 0, a, b, :, :, hidden][0,0] <= 1 + M * (1 - z1), name= f'f({y1}, 0, {a},{b},0,0) <= 1 + M * (1 - z1) = {1 + M * (1 - z1)}')

# m.addConstr(target[y1, 0, a, b, :, :, hidden][0,0] >= 2 - M * (1 - z2), name=f'f({y1}, 0, {a},{b},0,0) >= 2 - M * (1 - z2) = {2 - M * (1 - z2)}')
# m.addConstr(target[y1, 0, a, b, :, :, hidden][0,0] <= 2 + M * (1 - z2), name=f'f({y1}, 0, {a},{b},0,0) <= 2 + M * (1 - z2) = {2 + M * (1 - z2)}')

# m.addConstr(target[y1, 0, a, b, :, :, hidden][0,0] >= 3 - M * (1 - z3), name=f'f({y1}, 0, {a},{b},0,0) >= 3 - M * (1 - z3) = {3 - M * (1 - z3)}')
# m.addConstr(target[y1, 0, a, b, :, :, hidden][0,0] <= 3 + M * (1 - z3), name=f'f({y1}, 0, {a},{b},0,0) <= 3 + M * (1 - z3) = {3 + M * (1 - z3)}')

# # Ensure that exactly one of z1, z2, or z3 is 1
# m.addConstr(z1 + z2 + z3 == 1, name=f'f({y1}, 0, {a},{b},0,0) = 1 OR 2 OR 3')
# m.update()
# m.getConstrs()

# m.update()
# print(df_A_disc.loc[(df_A_disc.A == a) & (df_A_disc.B == b), 'Y1'].values[0], df_A_disc.loc[(df_A_disc.A == a) & (df_A_disc.B == b), 'Y1'].values[1])

# a, b, y1 = 1, 1, 1
# target[y1, 0, a, b, :, :, hidden][0,0] == 0 or 1
# m.addConstr(target[y1, 0, a, b, :, :, hidden][0, 0] == 0 or target[y1, 0, a, b, :, :, hidden][0, 0] == 1)



### impossible!

In [8]:
# concrete example:

df_full_conc = pd.DataFrame(columns=['occupation', 'wealth', 'education', 'age', 'litigousness', 'outsource-ness'])

# infinite run statistics:
for a,b,c,d in np.ndindex(3,2,2,2):
    df_full_conc = pd.concat([df_full_conc, pd.DataFrame({'occupation': [a], 'wealth': [b], 'education': [c], 'age': [d], 
                                              'litigousness': [np.minimum((0.5*a + 0.3*b)*c, d)], 
                                              'outsource-ness': [np.where(a == 0, 0, np.where(a == 1, 0.1*a + 0.2*b + 0.3*c + 0.4*d, a - d))]})], 
                                              ignore_index=True)

df_A_conc = pd.DataFrame(columns=['occupation', 'wealth', 'litigousness'])
df_B_conc = pd.DataFrame(columns=['wealth', 'education', 'age', 'outsource-ness'])
df_A_conc_no_filter = pd.DataFrame(columns=['occupation', 'wealth', 'litigousness'])
df_B_conc_no_filter = pd.DataFrame(columns=['wealth', 'education', 'age', 'outsource-ness'])
df_A_conc_no_filter = pd.concat([df_A_conc_no_filter, df_full_conc.loc[:, ['occupation', 'wealth', 'litigousness']]], ignore_index=True)
df_B_conc_no_filter = pd.concat([df_B_conc_no_filter, df_full_conc.loc[:, ['wealth', 'education', 'age', 'outsource-ness']]], ignore_index=True)

df_A_conc = pd.concat([df_A_conc, df_full_conc.loc[df_full_conc.age != 0, ['occupation', 'wealth', 'litigousness']]], ignore_index=True)
df_B_conc = pd.concat([df_B_conc, df_full_conc.loc[df_full_conc.occupation != 2, ['wealth', 'education', 'age', 'outsource-ness']]], ignore_index=True)


# display_side_by_side([df_full_conc, df_B_conc, df_A_conc], 
#                      ['df_full', 'df_B: df_full.occupation != 2', 'df_A: df_full.age != 0'])

display_side_by_side([df_B_conc, df_A_conc], 
                     ['df_B: df_full.occupation != 2', 'df_A: df_full.age != 0'])

# display_side_by_side([df_B_conc_no_filter, df_A_conc_no_filter], 
#                      ['df_B', 'df_A'])

,wealth,education,age,outsource-ness
0,0,0,0,0.0
1,0,0,1,0.0
2,0,1,0,0.0
3,0,1,1,0.0
4,1,0,0,0.0
5,1,0,1,0.0
6,1,1,0,0.0
7,1,1,1,0.0
8,0,0,0,0.1
9,0,0,1,0.5


In [9]:
# df_full_conc

# for a,b,c,d in np.ndindex(3,2,2,2):
#     df_full_conc = pd.concat([df_full_conc, pd.DataFrame({'occupation': [a], 'wealth': [b], 'education': [c], 'age': [d], 
#                                               'litigousness': [np.minimum((0.5*a + 0.3*b)*c, d)], 
#                                               'outsource-ness': [np.where(a == 0, 0, np.where(a == 1, 0.1*a + 0.2*b + 0.3*c + 0.4*d, a - d))]})], 
#                                               ignore_index=True)

In [10]:
# outsourceness 0.30000000000000004:
# occupation 1, wealth 1, education 0, age 0
# np.where(a == 0, 0, np.where(a == 1, 0.1*a + 0.2*b + 0.3*c + 0.4*d, a - d))]})

# outsourceness 0.7000000000000001
# occupation 1, wealth 1, education 0, age 1

# outsourceness 0.6000000000000001:
# occupation 1, wealth 1, education 1, age 0

a, b, c, d = 1, 1, 0, 0
print(np.where(a == 0, 0, np.where(a == 1, 0.1*a + 0.2*b + 0.3*c + 0.4*d, a - d)))
print(0.1 + 0.2)

0.30000000000000004
0.30000000000000004
